In [1]:
# (1) パッケージをインストールする（初回のみ）
install.packages("tidyverse")

# (2) ライブラリの読み出し
library("tidyverse")


Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

── Attaching packages ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.3     ✔ purrr   0.3.4
✔ tibble  3.1.2     ✔ dplyr   1.0.7
✔ tidyr   1.1.3     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.1

── Conflicts ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



In [2]:
# (3) データの読み込み
email_data <- read_csv("http://www.minethatdata.com/Kevin_Hillstrom_MineThatData_E-MailAnalytics_DataMiningChallenge_2008.03.20.csv")
head(email_data)


── Column specification ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
cols(
  recency = col_double(),
  history_segment = col_character(),
  history = col_double(),
  mens = col_double(),
  womens = col_double(),
  zip_code = col_character(),
  newbie = col_double(),
  channel = col_character(),
  segment = col_character(),
  visit = col_double(),
  conversion = col_double(),
  spend = col_double()
)




recency,history_segment,history,mens,womens,zip_code,newbie,channel,segment,visit,conversion,spend
<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>
10,2) $100 - $200,142.44,1,0,Surburban,0,Phone,Womens E-Mail,0,0,0
6,3) $200 - $350,329.08,1,1,Rural,1,Web,No E-Mail,0,0,0
7,2) $100 - $200,180.65,0,1,Surburban,1,Web,Womens E-Mail,0,0,0
9,5) $500 - $750,675.83,1,0,Rural,1,Web,Mens E-Mail,0,0,0
2,1) $0 - $100,45.34,1,0,Urban,0,Web,Womens E-Mail,0,0,0
6,2) $100 - $200,134.83,0,1,Surburban,0,Phone,Womens E-Mail,1,0,0


In [3]:
# (4) データの準備
## 女性向けメールが配信されたデータを削除したデータを作成
male_df <- email_data %>%
  filter(segment != "Womens E-Mail") %>% # 女性向けメールが配信されたデータを削除
  mutate(treatment = ifelse(segment == "Mens E-Mail", 1, 0)) #介入を表すtreatment変数を追加

In [4]:
head(male_df)

recency,history_segment,history,mens,womens,zip_code,newbie,channel,segment,visit,conversion,spend,treatment
<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
6,3) $200 - $350,329.08,1,1,Rural,1,Web,No E-Mail,0,0,0,0
9,5) $500 - $750,675.83,1,0,Rural,1,Web,Mens E-Mail,0,0,0,1
9,5) $500 - $750,675.07,1,1,Rural,1,Phone,Mens E-Mail,0,0,0,1
2,2) $100 - $200,101.64,0,1,Urban,0,Web,Mens E-Mail,1,0,0,1
4,3) $200 - $350,241.42,0,1,Rural,1,Multichannel,No E-Mail,0,0,0,0
3,1) $0 - $100,58.13,1,0,Urban,1,Web,No E-Mail,1,0,0,0


In [5]:
# (5) 集計による比較
## group_byとsummairseを使って集計
summary_by_segment <- male_df %>%
  group_by(treatment) %>% # データのグループ化
  summarise(conversion_rate = mean(conversion), # グループごとのconversionの平均
            spend_mean = mean(spend), # グループごとのspendの平均
            count = n()) # グループごとのデータ数


In [6]:
summary_by_segment
# メールでconversion_rateや利用金額が向上していることがわかる

treatment,conversion_rate,spend_mean,count
<dbl>,<dbl>,<dbl>,<int>
0,0.005726087,0.6527894,21306
1,0.012531093,1.4226165,21307


In [7]:
# (6) t検定を行う
## (a)男性向けメールが配信されたグループの購買データを得る
mens_mail <- male_df %>%
  filter(treatment == 1) %>%
  pull(spend) #特定の列をベクトルにするらしい

## (b)メールが配信されなかったグループの購買データを得る
no_mail <- male_df %>%
  filter(treatment == 0) %>%
  pull(spend)

In [8]:
## (a)(b)の平均の差に対して有意差検定を実行する
rct_ttest <- t.test(mens_mail, no_mail, var.equal = T)

In [9]:
rct_ttest


	Two Sample t-test

data:  mens_mail and no_mail
t = 5.3001, df = 42611, p-value = 1.163e-07
alternative hypothesis: true difference in means is not equal to 0
95 percent confidence interval:
 0.4851384 1.0545160
sample estimates:
mean of x mean of y 
1.4226165 0.6527894 


In [10]:
# (7) セレクションバイアスのあるデータの作成
## seedを固定する
set.seed(1)

## 条件に反応するサンプルの量を半分にする
obs_rate_c <- 0.5
obs_rate_t <- 0.5

## バイアスのあるデータの作成
biased_data <- male_df %>%
  mutate(obs_rate_c =
           ifelse( (history > 300) | (recency < 6) |
                     (channel == "Multichannel"), obs_rate_c, 1),
         obs_rate_t =
           ifelse( (history > 300) | (recency < 6) |
                     (channel == "Multichannel"), 1, obs_rate_t),
         random_number = runif(n = NROW(male_df))) %>% #runifは乱数的なやつらしい、ベクトルを返す
  filter( (treatment == 0 & random_number < obs_rate_c ) |
            (treatment == 1 & random_number < obs_rate_t) )

In [11]:
head(biased_data)

recency,history_segment,history,mens,womens,zip_code,newbie,channel,segment,visit,conversion,spend,treatment,obs_rate_c,obs_rate_t,random_number
<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
6,3) $200 - $350,329.08,1,1,Rural,1,Web,No E-Mail,0,0,0,0,0.5,1,0.2655087
9,5) $500 - $750,675.83,1,0,Rural,1,Web,Mens E-Mail,0,0,0,1,0.5,1,0.3721239
9,5) $500 - $750,675.07,1,1,Rural,1,Phone,Mens E-Mail,0,0,0,1,0.5,1,0.5728534
2,2) $100 - $200,101.64,0,1,Urban,0,Web,Mens E-Mail,1,0,0,1,0.5,1,0.9082078
4,3) $200 - $350,241.42,0,1,Rural,1,Multichannel,No E-Mail,0,0,0,0,0.5,1,0.2016819
5,1) $0 - $100,29.99,1,0,Surburban,0,Phone,Mens E-Mail,0,0,0,1,0.5,1,0.9446753


In [12]:
# (8) セレクションバイアスのあるデータで平均を比較
## group_byとsummairseを使って集計(Biased)
summary_by_segment_biased <- biased_data %>%
  group_by(treatment) %>%
  summarise(conversion_rate = mean(conversion),
            spend_mean = mean(spend),
            count = n())

head(summary_by_segment_biased)

treatment,conversion_rate,spend_mean,count
<dbl>,<dbl>,<dbl>,<int>
0,0.004977838,0.5483062,14665
1,0.013431794,1.5277526,17198


In [13]:
# (9) Rの関数であるt.testを使ってt検定を行う(Biased)
## (a)男性向けメールが配信されたグループの購買データを得る
mens_mail_biased <- biased_data %>%
  filter(treatment == 1) %>%
  pull(spend)

## (b)メールが配信されなかったグループの購買データを得る
no_mail_biased <- biased_data %>%
  filter(treatment == 0) %>%
  pull(spend)

## (a)(b)の平均の差に対して有意差検定を実行
rct_ttest_biased <- t.test(mens_mail_biased, no_mail_biased, var.equal = T)

In [14]:
rct_ttest_biased


	Two Sample t-test

data:  mens_mail_biased and no_mail_biased
t = 5.6708, df = 31861, p-value = 1.433e-08
alternative hypothesis: true difference in means is not equal to 0
95 percent confidence interval:
 0.6409145 1.3179784
sample estimates:
mean of x mean of y 
1.5277526 0.5483062 
